# Practicum 1: Edit regels voor categorische data

## 1. Introductie

<!--Tijdens het eerste practicum hebben jullie kennis kunnen maken met de eerste 2 V's die Big Data karakteriseert: *Volume* en *Velocity*.
Tijdens dit practicum focussen we op *Variety* en *Veracity*.-->

<!--Het variëteitsprobleem heeft betrekking op de grote verscheidenheid aan structuren waarin data te vinden zijn.
Zo kan het zijn dat data niet voldoen aan een voorgedefinieerd schema waardoor opslag in een relationele database zeer lastig is (zie vorig practicum).
Daarnaast is het mogelijk dat verschillende bronnen dezelfde informatie op verschillende manieren opslaan, waardoor uitwisseling, integratie en opslag veel werk vereist.
Tot slot is het mogelijk dat data semi- tot ongestructureerd zijn (bvb. tekstuele data of multimedia).-->

<!--Een andere grote vraag in de wereld van Big Data is hoe waarheidsgetrouw de data zijn.
De enorme overvloed aan data zorgt ervoor dat manuele controle van de kwaliteit lastig is.
Data-kwaliteit wordt vaak gedefinieerd aan de hand van verschillende dimensies: compleetheid (hoe volledig zijn de data?), consistentie (brengen de data binnen een systeem dezelfde informatie voort?), accuraatheid (hoe nauwkeurig weerspiegelen de data de werkelijkheid?), tijdigheid (hoe up-to-date zijn de data nog op dit moment?),...
Hoe beter de kwaliteit van de data, hoe nauwkeuriger en vollediger de analyses met behulp van deze data kunnen gebeuren.-->

Tijdens de theorielessen hebben jullie heel wat materiaal aangerijkt gekregen in verband met edit regels.
In de volgende twee oefeningenlessen gaan we praktisch aan de slag met edit regels voor respectievelijk categorische en continue data.

In dit eerste practicum bekijken we een kleine dataset die handelt over klinische studies.
Deze dataset bestaat uit attributen afkomstig van twee verschillende databronnen namelijk [EudraCT](https://eudract.ema.europa.eu/) en [ClinicalTrials.gov](https://clinicaltrials.gov/).
We proberen met behulp van het Fellegi-Holt framework inconsistenties op basis van edit regels op te sporen tussen deze databronnen.
In een later practicum is het dan de bedoeling om deze inconsistenties te gaan wegwerken met als doel de kwaliteit van de data te verbeteren.

Vooraleer we beginnen, maken we eerst even kennis met de pandas library, die een groot aantal functionaliteiten aanbiedt voor het verwerken en analyseren van gegevens.
Deze library zullen we gedurende dit en een aantal van de volgende practica gebruiken.

<!--In dit eerste practicum gaan we dieper in op de consistentie van data door data uit verschillende klinische studie-databronnen te gaan analyseren.
We zullen ons hierbij vooral richten op de consistentie van data over 2 databronnen, namelijk [EudraCT](https://eudract.ema.europa.eu/) en [ClinicalTrials.gov](https://clinicaltrials.gov/).
In eerste instantie zullen we stap voor stap concepten aanrijken om inconsistenties in en tussen databronnen op te sporen.
Daarnaast zullen we ook een aantal technieken bekijken om deze inconsistenties weg te werken met als doel de kwaliteit van de data te verbeteren.-->


## 2. Kennismaking met pandas

In dit practicum maken we gebruik van de [pandas](https://pandas.pydata.org/) library van Python.
Deze library wordt voornamelijk gebruikt voor het inlezen en analyseren van databronnen.
Om jullie enige voeling te doen krijgen met deze library, starten we met een aantal pandas-gebaseerde opdrachten.
Door middel van deze opdrachten maken jullie ook stap voor stap kennis met de dataset (clinical_trials_1.csv)

In [7]:
import pandas as pd

In [8]:
# Opdracht: lees de dataset 'clinical_trials_1.csv' in met behulp van pandas
dataset = pd.read_csv('clinical_trials_1.csv')

Zoals jullie kunnen zien is het zeer eenvoudig om een .csv bestand in te lezen met pandas.
Bij het inlezen van dit bestand wordt er een pandas DataFrame object aangemaakt dat simpelweg een tabel voorstelt met rijen en kolommen die overeenkomen met respectievelijk de rijen en kolommen van het ingeladen .csv bestand.
We bekijken dit DataFrame object even van naderbij.

In [14]:
# Opdracht: hoeveel rijen en kolommen er aanwezig zijn in het DataFrame object?
number_of_rows = len(dataset)
print("Number of rows: {:d}".format(number_of_rows))

number_of_columns = len(dataset.columns)
print("Number of columns: {:d}".format(number_of_columns))

Number of rows: 35945
Number of columns: 4


Om jullie een idee te geven hoe de dataset eruit ziet en welke waarden en kolommen er gepersisteerd zijn, is het mogelijk om de eerste rijen snel te bekijken.
Ook kunnen jullie per kolom uitprinten welke waarden er voorhanden zijn.

In [15]:
# Opdracht: print de eerste 10 rijen van de dataset uit
print("First ten rows of the dataset: ")
print(dataset.head(10))

First ten rows of the dataset: 
  open double_blind single_blind masking
0  NaN          NaN          NaN       0
1  Yes           No           No       2
2  Yes          Yes           No      >2
3  Yes           No           No       0
4   No          Yes           No      >2
5  Yes           No           No       0
6  Yes           No           No     NaN
7  Yes           No           No       0
8   No           No          Yes       1
9  Yes           No           No       0


In [16]:
# Opdracht: welke verschillende kolommen zijn er aanwezig in het DataFrame object en welke waarden zijn er te vinden per kolom?
print("All columns and column values: ")
print(dataset.columns)

All columns and column values: 
Index(['open', 'double_blind', 'single_blind', 'masking'], dtype='object')


Als je de vorige opdracht correct hebt uitgevoerd zal je zien dat er 4 kolommen te vinden zijn: 'open', 'double_blind', 'single_blind' en 'masking'.
De eerste 3 kolommen zijn afkomstig uit de [EudraCT](https://eudract.ema.europa.eu/) dataset en 'masking' is afkomstig uit de [ClinicalTrials.gov](https://clinicaltrials.gov/) dataset.
Elke rij specifieert de details van het ontwerp van een klinische studie die in beide datasets zijn opgenomen.

Vooraleer we verder gaan, geven we jullie een klein overzichtje van de betekenis van de kolommen.

* **open**: een studie waarvan zowel de onderzoekers als de proefpersonen weten welke behandeling er wordt toegepast
* **single_blind**: een studie waarvan maar 1 van beide partijen (meestal de onderzoekers) weten welke behandeling er wordt toegepast
* **double_blind**: een studie waarvan noch de onderzoekers, noch de proefpersonen weten welke behandeling er wordt toegepast
* **masking**: categorisch attribuut met betrekking tot de masking/blindness van de klinische studie (0 = Open, 1 = Single, 2 = Double, >2 = Triple/Quadruple)

Zoals jullie kunnen zien bevat elk attribuut ook een speciale waarde: NaN. Dit komt overeen met een `null`-waarde en geeft aan dat de data niet voorhanden zijn. Hoe groter het aantal `null`-waarden, hoe minder nauwkeurig analyses op basis van de data de werkelijkheid zullen reflecteren, en hoe nefaster voor een onderzoeker. De volgende opdracht zal jullie duidelijk maken hoeveel attribuutwaarden en rijen een `null`-waarde bevatten.



In [17]:
# Opdracht: hoeveel attribuutwaarden komen overeen met null
print("Number of null-values:")
print(dataset.isnull().sum(axis=0).sum())
# 5202

Number of null-values:
5202


In [21]:
# Opdracht: hoeveel rijen bevatten er een null-waarde
print("Number of rows containing a null-value:")
print(sum(1 for i in dataset.isnull().sum(axis=1) if i > 0))
# 2083

Number of rows containing a null-value:
2083


In [23]:
# Opdracht: print per attribuut uit hoeveel null-waaarden er zijn
print("Number of null-values per attribute:")
print(dataset.isnull().sum(axis=0))
# 1547,1622,1779,254

Number of null-values per attribute:
open            1547
double_blind    1622
single_blind    1779
masking          254
dtype: int64


Tot slot is het ook mogelijk om bepaalde kolommen en rijen te selecteren die voldoen aan opgegeven criteria.
Dit zorgt ervoor dat bepaalde data snel teruggevonden kan worden.

In [24]:
# Opdracht: print voor de eerste 10 rijen enkel het 'open' attribuut
print(dataset['open'].head(10))

0    NaN
1    Yes
2    Yes
3    Yes
4     No
5    Yes
6    Yes
7    Yes
8     No
9    Yes
Name: open, dtype: object


In [31]:
# Opdracht: print voor de eerste 10 rijen attributen 2 tot 4
print(dataset.head(10).iloc[:,1:4])

  double_blind single_blind masking
0          NaN          NaN       0
1           No           No       2
2          Yes           No      >2
3           No           No       0
4          Yes           No      >2
5           No           No       0
6           No           No     NaN
7           No           No       0
8           No          Yes       1
9           No           No       0


In [32]:
# Opdracht: print de data van de rij met index 1000
# open            Yes
# double_blind     No
# single_blind     No
# masking           0
print(dataset.iloc[1000])

open            Yes
double_blind     No
single_blind     No
masking           0
Name: 1000, dtype: object


In [33]:
# Opdracht: print de eerste 10 rijen waarin het 'single_blind' attribuut de waarde 'Yes' bevat
# rijen 8, 15, 24, 30, 32, 33, 54, 62, 88, 126
print(dataset[dataset['single_blind'] == 'Yes'].head(10))

    open double_blind single_blind masking
8     No           No          Yes       1
15    No           No          Yes       1
24    No           No          Yes      >2
30    No           No          Yes       2
32    No           No          Yes       1
33    No           No          Yes       1
54    No           No          Yes       1
62    No           No          Yes       0
88    No           No          Yes       1
126   No           No          Yes      >2


In [37]:
# Opdracht: print het aantal rijen waarin het 'single_blind' attribuut de waarde 'Yes' bevat en het 'masking' attribuut de waarde '1'
# 287
print(len(dataset[(dataset['single_blind'] == 'Yes') & (dataset['masking'] == '1')]))

287


In [46]:
# Opdracht: print de eerste 10 rijen die GEEN null-waarden bevatten
# 1, 2, 3, 4, 5, 7, 8, 9, 11, 12
print(dataset.nnull())

        open  double_blind  single_blind  masking
0      False         False         False     True
1       True          True          True     True
2       True          True          True     True
3       True          True          True     True
4       True          True          True     True
...      ...           ...           ...      ...
35940   True          True          True     True
35941   True          True          True     True
35942   True          True          True     True
35943   True          True          True     True
35944   True          True          True     True

[35945 rows x 4 columns]


## 3. Edit regels

Nu jullie kort kennis gemaakt hebben met de [pandas](https://pandas.pydata.org/) library van Python en met de eerste dataset die we gaan gebruiken tijdens dit practicum, is het tijd om de kwaliteit van deze dataset te gaan bepalen en te verbeteren.
We hebben namelijk reeds aangehaald dat de ingeladen dataset bestaat uit data gecombineerd uit 2 verschillende klinische studie-databronnen. In het vervolg onderzoeken we hoe consistent de data binnen (intra-consistentie) en tussen (inter-consistentie) de twee databronnen zijn.

Het basisprincipe om de kwaliteit van de dataset (of een deel van de dataset) vast te stellen en van waaruit wij gaan vertrekken is als volgt:
* In eerste instantie wordt er een set van regels opgesteld waaraan de data moeten voldoen. Deze regels leggen beperkingen vast op attribuutwaarden, attribuutwaarden-combinates, atttribuut-combinaties en/of de dataset in zijn geheel.
* Elke regel kan worden getest ten opzichte van het overeenkomstige object en resulteert in een booleaanse waarde ("ja, dit object voldoet aan de regel" of "neen, dit object voldoet niet aan de regel").
* Hoe minder regels er voldaan zijn, hoe lager de kwaliteit van het object. Indien alle regels voldaan zijn is de kwaliteit van het object perfect.

### 3.1 Definitie

In deze sectie zullen wij dit basisprincipe invullen door toepassing van *edit regels* op rijen van de gegeven dataset.
Edit regels zijn geïntroduceerd door Fellegi & Holt in hun paper: A Systematic Approach to Automatic Edit and Imputation (1976).
Een edit regel is een elegante manier om niet-toegestane attribuutwaarde-combinaties voor te stellen.

De algemene vorm van een edit regel is als volgt:
Een *edit regel* voor een dataset $R$ met attributen $a_1,\dots,a_k$ is een regel van het type $E_1 \times \dots \times E_k$ waarin elke $E_i$ een niet-lege deelverzameling is van $A_i$ ($A_i$ is het domein van alle mogelijke waarden voor $a_i$). Een edit regel legt met andere woorden een ruimte vast binnen $A_1 \times \dots \times A_k$ van niet-toegestane rijen.
Indien een rij voorkomt in deze ruimte voldoet deze rij **NIET** aan de regel, anders wel.

Voor de eerste dataset hebben we reeds een verzameling van 8 edit regel vastgelegd. Deze zijn te vinden in onderstaande tabel.

| Regel | open      | single_blind | double_blind | masking      |
|-------|-----------|--------------|--------------|--------------|
| 1     | Yes       | dom(single)  | Yes          | dom(masking) |
| 2     | dom(open) | Yes          | Yes          | dom(masking) |
| 3     | Yes       | Yes          | dom(double)  | dom(masking) |
| 4     | dom(open) | No           | dom(double)  | 1            |
| 5     | dom(open) | dom(single)  | No           | 2            |
| 6     | dom(open) | dom(single)  | Yes          | 0            |
| 7     | dom(open) | Yes          | dom(double)  | 0            |
| 8     | dom(open) | dom(single)  | Yes          | 1            |

Als voorbeeld bekijken we even regel 1: {Yes} $\times$ dom(single) $\times$ {Yes} $\times$ dom(masking).
Deze regel legt vast dat open = 'Yes' en double_blind = 'Yes' **NIET** samen kunnen voorkomen, welke waarde de andere twee attributen ook aannemen (dom(X) verwijst naar het hele domein van attribuut X).
Dit houdt semantisch ook steek, aangezien het niet mogelijk is dat een studie zowel 'open' als 'double-blind' is.
Bekijk even de andere edit regels en probeer ze semantisch te interpreteren vooraleer je verdergaat.


In [ ]:
# Opdracht: tel het aantal rijen uit de dataset die niet voldoen aan edit regel 1 uit bovenstaande tabel
failing_rows = # ...
print("Number of rows failing edit rule 1: {:d}".format(failing_rows))

In onderstaand stuk code hebben wij voor jullie reeds de 8 gegeven edit regels geïnitialiseerd. Deze zullen jullie nodig hebben tijdens de rest van dit practicum. Indien een attribuut niet gegeven is in een edit regel, betekent dit dat binnen deze edit regel dit attribuut het volledige attribuut-domein beslaat.

In [ ]:
edit_rules = [
                {
                    'open' : 'Yes',
                    'double_blind' : 'Yes'
                },
                {
                    'single_blind' : 'Yes',
                    'double_blind' : 'Yes'
                },
                {
                    'open' : 'Yes',
                    'single_blind' : 'Yes'
                }, 
                {
                    'single_blind' : 'No',
                    'masking' : '1'
                },
                {
                    'double_blind' : 'No',
                    'masking' : '2'
                },
                {
                    'double_blind' : 'Yes',
                    'masking' : '0'
                },
                {
                    'single_blind' : 'Yes',
                    'masking' : '0'
                },
                {
                    'double_blind' : 'Yes',
                    'masking' : '1'
                }
             ]

print("Edit rules: ")
for edit_rule in edit_rules:
    print(edit_rule)

In [ ]:
# Opdracht: implementeer een algoritme dat in de violations dictionary per rijnummer opslaat welke edit regels deze rij schendt indien dit er meer dan 0 zijn.

def get_violated_rows(dataset, edit_rules):
    violations = {}

    for index, row in dataset.iterrows():

        # debug info
        if index % 5000 == 0:
            print(index)
            
        # TO IMPLEMENT...
            
    return violations

violations = get_violated_rows(dataset, edit_rules)
print("number of violated rows: {:d}".format(len(violations.keys())))

# 1222

In [ ]:
# Opdracht: valideer voor de eerste rij die opgeslagen is in de violations dictionary of je algoritme effectief het juiste resultaat teruggeeft
row = next(iter(violations))
print(dataset.iloc[row])

for edit_rule_index in violations[row]:
    print(edit_rules[edit_rule_index - 1])

### 3.2 Complete set van edit regels

In de vorige sectie hebben we gekeken naar welke rijen niet voldoen aan de gegeven set van edit regels.
In deze sectie gaan we dieper in op localiseren van de mogelijke fouten.

Het is eenvoudig te zien dat een rij enkel gerepareerd kan worden als de attribuutwaarde van een attribuut dat *entert* in een edit regel die niet voldaan is wordt aangepast (entert = de set van waarden voor dit attribuut in deze edit regel beslaan **NIET** het volledige domein)

Neem als voorbeeld de volgende rij:

| open      | single_blind | double_blind | masking      |
|-----------|--------------|--------------|--------------|
| Yes       | No           | Yes          | 2            |

Zoals je kan zien voldoet deze regel niet aan de edit regel 1: {Yes} $\times$ dom(single) $\times$ {Yes} $\times$ dom(masking).
In deze edit regel zien we dat de attributen 'open' en 'double_blind' enteren aangezien de waarden voor deze attributen een echte deelverzameling vormen van de domeinen.
Om ervoor te zorgen dat bovenstaande rij voldoet aan deze edit regel is het dus noodzakelijk om ofwel de waarde voor het attribuut 'open' ofwel de waarde voor attribuut 'double_blind' aan te passen.

Als we de waarde van het attribuut 'open' veranderen naar 'No' krijgen we volgende rij:

| open      | single_blind | double_blind | masking      |
|-----------|--------------|--------------|--------------|
| No        | No           | Yes          | 2            |

Het is duidelijk dat deze rij voldoet aan edit regel 1.
Bovendien voldoet deze rij ook aan alle andere gegeven edit regels.

Stel daarentegen dat we de waarde van het attribuut 'double_blind' veranderen naar 'No' in de oorspronkelijke rij.
Dan verkrijgen we de volgende rij:

| open      | single_blind | double_blind | masking      |
|-----------|--------------|--------------|--------------|
| Yes       | No           | No           | 2            |

Ook deze rij voldoet aan edit regel 1.
Als we deze rij opnieuw controleren voor de hele gegeven verzameling zien we dat deze rij NIET voldoet aan edit regel 5: dom(open) $\times$ dom(single) $\times$ {No} $\times$ {2}.
Dit geeft dus opnieuw probleem.

De reden dat dit opnieuw een probleem geeft is omdat edit regels 1 & 5 niet onafhankelijk zijn van elkaar.
Aangezien edit regel 1 vermeldt dat 'open' = 'Yes' en double_blind = 'Yes' niet samen mogen voorkomen, mag 'open' = 'Yes' dus enkel met 'double_blind' = 'No' voorkomen.
Edit regel 5 vertelt ons dat 'double_blind' = 'No' niet met 'masking' = '2' mag voorkomen.
Dit impliceert dus ook dat open = 'Yes' niet met masking = '2' mag voorkomen.
We kunnen dus een nieuwe geïmpliceerde edit regel afleiden uit deze twee regels.

Formeel definiëren we dit als volgt:

Neem een verzameling $\mathbb{E}$ bestaande uit minstens 2 edit regels (contributing set) en een attribuut $g$ (generator-attribuut).
Construeer een nieuwe edit regel $E_1^* \times \cdots \times E_k^*$ zo dat $E_i^*$ bestaat uit:
* de unie van attribuutwaarden van de regels in $\mathbb{E}$ voor attribuut $g$
* de doorsnede van attribuutwaarden van de regels in $\mathbb{E}$ voor alle andere attributen

Als geen enkele $E_i^*$ leeg is, is $E_1^* \times \cdots \times E_k^*$ een *geïmpliceerde regel*.

Indien bovendien het attribuut $g$ entert in elke regel in $\mathbb{E}$ en $g$ niet entert in de geïmpliceerde regel, dan is de geïmpliceerde regel *essentieel nieuw*.

De verzameling van gegeven edit regels tesamen met de essentieel nieuwe edit regels wordt de *complete set* van edit regels genoemd.
Deze complete set zal, zoals bewezen door Fellegi & Holt, steeds een oplossing bieden voor het localiseren van de fout en zal dus ook steeds een mogelijke reparatie voorstellen.

Probeer de definitie van een essentieel nieuw geïmpliceerde edit regel te begrijpen vooraleer verder te gaan.
Pas de definitie eens toe op bovenstaand voorbeeld.


Zoals jullie uit bovenstaande beschrijving begrijpen is het belangrijk om de complete set van edit regels te vinden om het fout-localisatie probleem op te lossen. 
Doe dit manueel voor de gegeven set met behulp van het Field Code Forest algoritme, geïntroduceerd in de theorielessen

In [ ]:
# Opdracht: vind MANUEEL mbv het Field Code Forest algoritme alle essentieel nieuwe edit regels en voeg ze toe aan de gegeven set van edit regels (de eerste is reeds gegeven in bovenstaande beschrijving)
edit_rules.append({'open' : 'Yes', 'masking' : '2'})
#edit_rules.append(...)

print("Edit rules: ")
for edit_rule in edit_rules:
    print(edit_rule)

In [ ]:
# Opdracht: voeg opnieuw aan de violations dictionary alle rijen toe samen met de regels waaraan ze niet voldoen door uitvoering van het algoritme dat jullie eerder hebben geïmplementeerd

violations = get_violated_rows(dataset, edit_rules)
print("number of violated rows: {:d}".format(len(violations.keys())))
# 1224

### 3.3 Fout-localisatie

Nadat de complete set van edit regels is gegenereerd en per rij de niet-voldane edit regels zijn geïdentificeerd is het mogelijk om per rij die niet voldoet aan alle regels de fout te gaan localiseren.
Dit kan gedaan worden door per rij de minimale covering set van attributen te gaan detecteren. 
Een minimale covering set moet voldoen aan 2 eigenschappen:
* De set is covering: elke niet-voldane edit rule heeft minstens 1 attribuut uit deze set dat entert
* De set is minimaal: bij het weglaten van een attribuut verdwijnt de covering eigenschap van dit attribuut

Stel dat de dataset volgende rij bevat:

| open      | single_blind | double_blind | masking      |
|-----------|--------------|--------------|--------------|
| No        | Yes          | Yes          | 0            |

In de oorspronkelijke (dus niet de complete) verzameling van edit regels zijn er 3 regels waaraan deze rij niet voldoet, namelijk

| Regel | open      | single_blind | double_blind | masking      |
|-------|-----------|--------------|--------------|--------------|
| 2     | dom(open) | Yes          | Yes          | dom(masking) |
| 6     | dom(open) | dom(single)  | Yes          | 0            |
| 7     | dom(open) | Yes          | dom(double)  | 0            |

De minimale sets van attributen die deze regels covert is ofwel {'single_blind', 'double_blind'}, ofwel {'single_blind', 'masking'} ofwel {'double_blind', 'masking'}. 
Elke niet-voldane regel heeft namelijk minstens 1 attribuut uit deze sets dat entert en deze sets zijn minimaal (je kan er geen attributen uit weglaten zodat de set nog steeds covering is).
Uit deze minimale sets kiezen we er een die we gaan gebruiken om de rij te repareren.

In [ ]:
# Opdracht: implementeer een algoritme dat voor elke rij eerst de 1-sets van attributen test, dan de 2-sets,... tot er een minimale covering set is gevonden.

import itertools

def get_minimal_covering_set(violated_rules):
    #...
    
minimal_covering_sets = {}
                
for row in violations:
    minimal_covering_sets[row] = get_minimal_covering_set(violations[row])     

**Ter info**

Het is mogelijk (zoals in bovenstaand voorbeeld) dat er meerdere minimale covering sets van attributen worden gevonden per rij.
Om te kiezen welke covering set je dan neemt kan je gaan kijken naar het aantal rijen (de frequenties) die dezelfde attribuutwaarden hebben als de gegeven rij *zonder* de attribuutwaarden van de attributen in de minimale covering set in acht te nemen.
Rekening houdend met deze distributie trek je dan willekeurig een minimale covering set die je zal gebruiken om te imputeren.